# Closed Domain Chatbot Maker

![chatbot-gig](https://media2.giphy.com/media/S60CrN9iMxFlyp7uM8/giphy.gif)

In [16]:
import io
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, tokenizer_from_json

## Training Data

In [ ]:
with open('tags.txt', encoding='utf8') as file_in:
    X = []
    Y = []
    for line in file_in:
        Y.append(line.strip().split(' ')[-1])
        X.append(' '.join(line.strip().split(' ')[:-1]))
Y = pd.get_dummies(pd.DataFrame(np.asarray(Y).astype(np.float)).rename(columns={0: 'label'})['label'], prefix='label_').values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Tokenizer

In [19]:
vocab_size = 3000
embed_size = 50
max_len = 100

tokenizer = Tokenizer(num_words=vocab_size,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      oov_token="<OOV>")

tokenizer.fit_on_texts(x_train)
training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(
    training_sequences, maxlen=max_len, truncating='post')


tokenizer_json = tokenizer.to_json()
with io.open('pre_trained_aira\\aira_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

## Bi-LSTM

In [22]:
inputs = tf.keras.Input(shape=(None,), dtype="int32")
x = tf.keras.layers.Embedding(input_dim=vocab_size,
                              output_dim=embed_size,
                              input_length=max_len)(inputs)

x = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)

outputs = tf.keras.layers.Dense(142, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy '])

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
model.summary()

model.fit(training_padded,
          y_train,
          validation_split = 0.2,
          epochs=50,
          batch_size=5,
          verbose=2)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, maxlen=100, truncating='post')

test_loss_score, test_acc_score = model.evaluate(test_padded, y_test)

print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')
model.save('pre_trained_aira\pre_trained_aira_lstm.h5')

Version:  2.10.0
Eager mode:  True
GPU is available
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_10 (Embedding)    (None, None, 50)          150000    
                                                                 
 bidirectional_12 (Bidirecti  (None, None, 128)        58880     
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dense_12 (Dense)            (None, 142)               18318     
                                                                 
Total p

ValueError: in user code:

    File "c:\Users\CWLINK\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\CWLINK\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\CWLINK\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\CWLINK\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 998, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\CWLINK\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1092, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, s

## Transformer

In [15]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    vocab_size = 3000
    embed_size = 50
    max_len = 100
    inputs = tf.keras.Input(shape=input_shape, dtype="int32")
    x = tf.keras.layers.Embedding(input_dim=vocab_size,
                              output_dim=embed_size,
                              input_length=max_len)(inputs)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(142, activation="softmax")(x)
    return keras.Model(inputs, outputs)

input_shape = (training_padded.shape[1])

model = build_model(
    input_shape,
    head_size=256,
    num_heads=6,
    ff_dim=4,
    num_transformer_blocks=6,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.2,
)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
model.summary()

model.fit(training_padded,
          y_train,
          validation_split = 0.2,
          epochs=100,
          batch_size=5,
          verbose=1)

test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, maxlen=100, truncating='post')

test_loss_score, test_acc_score = model.evaluate(test_padded, y_test)

print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')
model.save('pre_trained_aira\pre_trained_aira_transformer.h5')

Version:  2.10.0
Eager mode:  True
GPU is available
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding_7 (Embedding)        (None, 100, 50)      150000      ['input_8[0][0]']                
                                                                                                  
 layer_normalization_24 (LayerN  (None, 100, 50)     100         ['embedding_7[0][0]']            
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_12 (Multi  (None, 